In [150]:
import numpy as np
import matplotlib.pyplot as plt
import torch
plt.style.use('dark_background')

In [ ]:
L_opt = 300*(1e-12)
w = 2*np.pi*(20e9)
Z_L = 20+1j*(w*L_opt) # (target) load impedance
Z_S = 20 # fixed source impedance

def computeGamma(Z,Z0=50): z = Z/Z0; return ((z-1)/(z+1))
def transformedImpedance(match__Lout_pH, match__Cout_fF=0, freq=20e9):#,match__Cout):
    w = 2*np.pi*freq
    # Series inductor, Shunt capacitor (Source --> Load)  
    cap_b = 1j*w*torch.abs(match__Cout_fF)*(1e-15)
    ind_x = 1j*w*torch.abs(match__Lout_pH)*(1e-12)
    A = 1 + ind_x*cap_b; B = ind_x; C = cap_b; D=1
    Z_S_transformed = (D*Z_S+B)/(C*Z_S+A)
    # Z_S_transformed = Z_S + 1j*w*match__Lout_pH*(1e-12)
    return Z_S_transformed

def computeError(match__Lout_pH,match__Cout_fF):#,match__Cout):
    # These work
    # impedance_error = torch.abs(match__Lout_pH - 300)**2
    # impedance_error = torch.abs(w*match__Lout_pH - w*300)**2
    
    # Z_S_transformed = Z_S + 1j*w*match__Lout_pH
    Z_S_transformed = transformedImpedance(match__Lout_pH,match__Cout_fF)
    impedance_error = torch.abs(Z_S_transformed - Z_L)**2

    return impedance_error

In [183]:
match__Lout = torch.tensor(10.9, requires_grad=True)
match__Cout_fF = torch.tensor(10.9, requires_grad=True)

optimizer = torch.optim.Adam([match__Lout,match__Cout_fF], lr=10) # Adaptive algorithm, better
for i in range(500):
    optimizer.zero_grad()
    f = computeError(match__Lout,match__Cout_fF)
    f.backward()
    optimizer.step()
    if i % 100 == 0:
        # print(f":Iteration {i}:  Lout = {match__Lout.item()}, error = {f.item()}")
        print(f":Iteration {i}:  Lout = {match__Lout.item()}, Cout = {match__Cout_fF}, error = {f.item()}")

:Iteration 0:  Lout = 20.89999771118164, Cout = 0.9000015258789062, error = 1359.944580078125
:Iteration 100:  Lout = 297.9532165527344, Cout = -0.3923933804035187, error = 0.05748753622174263
:Iteration 200:  Lout = 300.00152587890625, Cout = 0.0034282454289495945, error = 1.1692936823237687e-06
:Iteration 300:  Lout = 299.9999694824219, Cout = -3.6860954423900694e-05, error = 3.2741809263825417e-11
:Iteration 400:  Lout = 299.9999694824219, Cout = -1.4601042494177818e-05, error = 1.8189894035458565e-11
